- ref. 

In [16]:
# load data
import pandas as pd, numpy as np
pd.set_option("display.width", 80)
pd.set_option("display.max_rows", 20)
import seaborn as sns, matplotlib.pyplot as plt
import multiprocessing as mp
n_cpus = mp.cpu_count()

train_df = pd.read_csv("./train.csv")   # 42000
# train_df = pd.read_csv("./train.csv", nrows=1000)
test_df = pd.read_csv("./test.csv")   # 28000
train_num = train_df.shape[0]
test_num = test_df.shape[0]
train_x = train_df.drop(['label'], axis=1)
train_y = train_df["label"]
test_x = test_df
all_df = pd.concat((train_x, test_df))

dfd = {'train': train_df, 'test': test_df, 'all': all_df}
for name,df in dfd.items():
    print(name, df.shape)
    print(df.head())

train (42000, 785)
   label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      1       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      1       0       0       0       0       0       0       0       0   
3      4       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   

   pixel8    ...     pixel774  pixel775  pixel776  pixel777  pixel778  \
0       0    ...            0         0         0         0         0   
1       0    ...            0         0         0         0         0   
2       0    ...            0         0         0         0         0   
3       0    ...            0         0         0         0         0   
4       0    ...            0         0         0         0         0   

   pixel779  pixel780  pixel781  pixel782  pixel783  
0         0         0         0

In [17]:
# check data
for name,df in dfd.items():
    print(name)
    print(df.info())
    #print(df.describe())   # doesn't show a lot of information

train
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB
None
test
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 167.5 MB
None
all
<class 'pandas.core.frame.DataFrame'>
Int64Index: 70000 entries, 0 to 27999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 419.2 MB
None


In [18]:
# check null
for name,df in dfd.items():
    print(name)
    isnull_ratio = df.isnull().sum()/df.isnull().count()
    print(isnull_ratio[isnull_ratio != 0].sort_values(ascending=False))

Series([], dtype: float64)


Series([], dtype: float64)
all


Series([], dtype: float64)
test


train


In [19]:
# check correlation (takes time)
# -> pixel436 has high correlation with label.. so what?
# correlations = train_df.astype(float).corr()['label'].sort_values(ascending=False)
# print(correlations.head())
# print(correlations.tail())

In [20]:
# check skewness (takes time)
# -> pixel367 has high skewness, but log/boxcox transform doesn't help
# from scipy.stats import skew
# skewness = all_df.apply(lambda x: skew(x)).sort_values(ascending=False)
# print(skewness.head())
# print(skewness.tail())
# all_df['pixel367'].hist(bins=50)
# plt.show()

In [21]:
# grid search (takes time)
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from mlxtend.classifier import StackingClassifier
from sklearn.base import clone

train_x = all_df[:train_num]
test_x = all_df[train_num:]

# models = []
# C_list, gamma_list, degree_list = np.logspace(-2, 0, 3), np.logspace(-3, -1, 3), np.linspace(1, 3, 3)
# grid = [
#     {'C': C_list, 'kernel': ['poly'], 'gamma': gamma_list, 'degree': degree_list},
#     # {'C': C_list, 'kernel': ['sigmoid'], 'gamma': gamma_list},
#     # {'C': C_list, 'kernel': ['linear']},
#     # {'C': C_list, 'kernel': ['rbf'], 'gamma': gamma_list},
#     ]
# models.append(('svm', GridSearchCV(SVC(), grid, cv=3, n_jobs=n_cpus)))
# grid = [{'n_estimators': [1000], 'max_depth': [6,7,8], 'min_samples_leaf': [2,3], 'max_features' : ['sqrt']}]
# models.append(('random_forest', GridSearchCV(RandomForestClassifier(), grid, cv=3, n_jobs=n_cpus)))
# models.append(('extra_trees', GridSearchCV(ExtraTreesClassifier(), grid, cv=3, n_jobs=n_cpus)))
# grid = {'n_estimators': [1000], 'learning_rate' : [0.25, 0.5, 0.75]}
# models.append(('ada_boost', GridSearchCV(AdaBoostClassifier(), grid, cv=3, n_jobs=n_cpus)))
# grid = {'n_estimators': [1000], 'max_depth': [6,7,8], 'min_samples_leaf': [2,3]}
# models.append(('gradient_boosting', GridSearchCV(GradientBoostingClassifier(), grid, cv=3, n_jobs=n_cpus)))
# grid = {'max_iter': [1000], 'tol': [0.001]}
# models.append(('perceptron', GridSearchCV(Perceptron(max_iter=1000, tol=0.001), grid, cv=3, n_jobs=n_cpus)))
# models.append(('sgd_classifier', GridSearchCV(SGDClassifier(max_iter=1000, tol=0.001), grid, cv=3, n_jobs=n_cpus)))
# grid = {'n_neighbors': [2,3,4]}
# models.append(('k-nearest_neighbors', GridSearchCV(KNeighborsClassifier(), grid, cv=3, n_jobs=n_cpus)))
# grid = {}
# models.append(('logistic_regression', GridSerchCV(LogisticRegression(), grid, cv=3, n_jobs=n_cpus)))
# models.append(('gaussian_naive bayes', GridSearchCV(GaussianNB(), grid, cv=3, n_jobs=n_cpus)))
# models.append(('linear_svc', GridSearchCV(LinearSVC(), grid, cv=3, n_jobs=n_cpus)))
# models.append(('decision_tree', GridSearchCV(DecisionTreeClassifier(), grid, cv=3, n_jobs=n_cpus)))

# params = {}
# for name,model in models:
#     model.fit(train_x, train_y)
#     print(name, model.best_score_, model.best_params_)
#     params[name] = model.best_params_
#     # results = model.cv_results_
#     # for mean, std, params in zip(results['mean_test_score'], results['std_test_score'], results['params']):
#     #     print("{:0.3f} (+/-{:0.03f} for {}".format(mean, std, params))

# from the result of nrows=20000
params = {
    'k-nearest_neighbors': {'n_neighbors': 3},
    'sgd_classifier': {'max_iter': 1000, 'tol': 0.001},
    'perceptron': {'max_iter': 1000, 'tol': 0.001},
    'gradient_boosting': {'max_depth': 6, 'min_samples_leaf': 3, 'n_estimators': 1000},
    'ada_boost': {'learning_rate': 0.25, 'n_estimators': 1000},
    'extra_trees': {'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'n_estimators': 1000},
    'random_forest': {'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'n_estimators': 1000},
    'svm': {'C': 0.01, 'degree': 2.0, 'gamma': 0.001, 'kernel': 'poly'},
    'logistic_regression': {},
    'gaussian_naive bayes': {},
    'linear_svc': {},
    'decision_tree': {},
    }

svm_model = SVC(**params['svm'])
bagging_svm_model = BaggingClassifier(base_estimator=clone(svm_model),
                                      n_estimators=10, max_samples=0.9, max_features=0.2)
et_model = ExtraTreesClassifier(**params['extra_trees'])
bagging_et_model = BaggingClassifier(base_estimator=clone(et_model),
                                     n_estimators=10, max_samples=0.9, max_features=0.2)
knn_model = KNeighborsClassifier(**params['k-nearest_neighbors'])
bagging_knn_model = BaggingClassifier(base_estimator=clone(knn_model),
                                      n_estimators=10, max_samples=0.9, max_features=0.2)
lr_model = LogisticRegression(**params['logistic_regression'])
stacking_model = StackingClassifier(classifiers=[clone(svm_model), clone(et_model), clone(knn_model)],
                                    meta_classifier=clone(lr_model))
models = [
    ('svm', svm_model),
    # ('bagging_svm', bagging_svm_model),
    ('extra_trees', et_model),
    # ('bagging_extra_trees', bagging_et_model),
    ('random_forest', RandomForestClassifier(**params['random_forest'])),
    ('ada_boost', AdaBoostClassifier(**params['ada_boost'])),
    ('gradient_boosting', GradientBoostingClassifier(**params['gradient_boosting'])),
    ('perceptron', Perceptron(**params['perceptron'])),
    ('sgd_classifier', SGDClassifier(**params['sgd_classifier'])),
    ('k-nearest_neighbors', knn_model),
    # ('bagging_k-nearest_neighbors', bagging_knn_model),
    ('logistic_regression', lr_model),
    ('gaussian_naive bayes', GaussianNB(**params['gaussian_naive bayes'])),
    ('linear_svc', LinearSVC(**params['linear_svc'])),
    ('decision_tree', DecisionTreeClassifier(**params['decision_tree'])),
    # ('stacking', stacking_model),
    ]

In [22]:
# cross validation
from sklearn.model_selection import KFold, cross_validate
from multiprocessing import Manager
from sklearn.externals.joblib import Parallel, delayed

k_fold = KFold(n_splits=3)
with Manager() as manager:
    results = manager.list()
    def validate_one_model(name, model):
         # Multiprocessing-backed parallel loops cannot be nested
         scores = cross_validate(model, train_x, train_y, cv=k_fold, scoring='accuracy')
         result = {}
         result['model'] = name
         result['test_mean'] = np.mean(scores['test_score'])
         result['test_std'] = np.std(scores['test_score'])
         result['train_mean'] = np.mean(scores['train_score'])
         result['train_std'] = np.std(scores['train_score'])
         results.append(result)
    Parallel(n_jobs=n_cpus)(delayed(validate_one_model)(name, model) for name,model in models)
    results_merged = {'model': [], 'test_mean': [], 'test_std': [], 'train_mean': [], 'train_std': []}
    for result in results:
        for k,v in result.items():
            results_merged[k].append(v)
    print(pd.DataFrame(results_merged).sort_values(by='test_mean', ascending=False))

                   model  test_mean  test_std  train_mean  train_std
7                    svm   0.974429  0.001222    1.000000   0.000000
11     gradient_boosting   0.966881  0.000897    1.000000   0.000000
9    k-nearest_neighbors   0.965429  0.001125    0.981595   0.000276
4          random_forest   0.926714  0.002734    0.942488   0.001532
3            extra_trees   0.916786  0.003062    0.929190   0.000992
10   logistic_regression   0.901405  0.000994    0.946464   0.000582
1         sgd_classifier   0.875095  0.009214    0.897393   0.013037
0             perceptron   0.858262  0.006284    0.874333   0.004659
6             linear_svc   0.850429  0.004476    0.880464   0.002844
5          decision_tree   0.845929  0.000498    1.000000   0.000000
8              ada_boost   0.737548  0.032553    0.743036   0.027985
2   gaussian_naive bayes   0.562619  0.006971    0.565512   0.007930


In [23]:
# learn, predict and report
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.externals.joblib import Parallel, delayed, dump
test_id = np.arange(1, test_num+1)
results = {}
def process_one_model(name, model):
    model.fit(train_x, train_y)
    predict_y = model.predict(train_x)
    output = model.predict(test_x)
    results[name] = output
    submit = pd.DataFrame(data={'ImageId':test_id, 'Label':output})
    submit.to_csv('{}_submit.csv'.format(name), index=False)
    #dump(model, '{}.learn'.format(name))
    print(name)
    print(confusion_matrix(train_y, predict_y))
    print(classification_report(train_y, predict_y))
Parallel(n_jobs=n_cpus)(delayed(process_one_model)(name, model) for name,model in models)

[None, None, None, None, None, None, None, None, None, None, None, None]

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      4132
          1       1.00      1.00      1.00      4684
          2       1.00      1.00      1.00      4177
          3       1.00      1.00      1.00      4351
          4       1.00      1.00      1.00      4072
          5       1.00      1.00      1.00      3795
          6       1.00      1.00      1.00      4137
          7       1.00      1.00      1.00      4401
          8       1.00      1.00      1.00      4063
          9       1.00      1.00      1.00      4188

avg / total       1.00      1.00      1.00     42000


[[4132    0    0    0    0    0    0    0    0    0]
 [   0 4684    0    0    0    0    0    0    0    0]
 [   0    0 4177    0    0    0    0    0    0    0]
 [   0    0    0 4351    0    0    0    0    0    0]
 [   0    0    0    0 4072    0    0    0    0    0]
 [   0    0    0    0    0 3795    0    0    0    0]
 [   0    0    0    0    0    0 4137    0    0    0]
 [   0    0    0    0    0    0    0 4401    0    0]
 [   0    0    0    0    0    0    0    0 4063    0]
 [   0    0    0    0    0    0    0    0    0 4188]]

gradient_boosting


             precision    recall  f1-score   support

          0       0.98      1.00      0.99      4132
          1       0.97      1.00      0.99      4684
          2       0.99      0.98      0.99      4177
          3       0.98      0.98      0.98      4351
          4       0.99      0.98      0.98      4072
          5       0.98      0.98      0.98      3795
          6       0.99      0.99      0.99      4137
          7       0.98      0.98      0.98      4401
          8       1.00      0.96      0.98      4063
          9       0.97      0.98      0.98      4188

avg / total       0.98      0.98      0.98     42000


...

[[4029    0   12   21    3   13   29    4   17    4]
 [   0 4453   21   76    1   25   10   17   79    2]
 [  18   11 3623  237   24   27   92   60   79    6]
 [   7    3   78 4106    0   64   18   24   31   20]
 [   8   17   30   59 3611    8   55   77   72  135]
 [  53    8   22  373   30 3075   45   24  120   45]
 [  22    2   17    0    9   68 4005    2   10    2]
 [  20    3   29   81   20   10    4 4191    7   36]
 [  25   32   38  553    8  132   31   47 3155   42]
 [  20    7   32  217  110   37    1  535   90 3139]]

sgd_classifier


             precision    recall  f1-score   support

          0       0.93      0.98      0.95      4132
          1       0.96      0.97      0.96      4684
          2       0.98      0.73      0.84      4177
          3       0.84      0.86      0.85      4351
          4       0.88      0.94      0.91      4072
          5       0.74      0.86      0.80      3795
          6       0.96      0.93      0.94      4137
          7       0.91      0.93      0.92      4401
          8       0.74      0.82      0.78      4063
          9       0.91      0.78      0.84      4188

avg / total       0.89      0.88      0.88     42000


[[4040    0    1   15   12   25    8    4   25    2]
 [   1 4538    5   18    3   46    4    9   52    8]
 [  84   74 3057  247   82   84   56   72  416    5]
 [  19   17   22 3721    8  251    6   24  230   53]
 [   8    7    3    6 3824   20   13   14   84   93]
 [  51    5    2  232   39 3267   67    9   88   35]
 [  54   11    8    4   51  114 3864   10   21    0]
 [   9   15   14   28   38   31    1 4095   62  108]
 [  45   50    7  106   30  410   23   21 3346   25]
 [  19    8    3   66  271  152    1  244  178 3246]]

perceptron


In [24]:
# neural network with tensorflow (skflow -> contrib.learn -> estimator)
# import tensorflow as tf
# feature_columns = [tf.feature_column.numeric_column(k) for k in train_x.columns]
# tf_model = tf.estimator.DNNClassifier(hidden_units=[1000], n_classes=10, feature_columns=feature_columns)
# input_fn_train = tf.estimator.inputs.pandas_input_fn(x=train_x, y=train_y, shuffle=False)
# tf_model.train(input_fn=input_fn_train, steps=100)
# tf_model.evaluate(input_fn=input_fn_train, steps=10)
# #scores = cross_validate(tf_model, train_x, train_y, cv=k_fold, scoring='accuracy')   # throws an error
# input_fn_predict = tf.estimator.inputs.pandas_input_fn(x=test_x, shuffle=False)
# generator = tf_model.predict(input_fn=input_fn_predict)
# output = [d['class_ids'][0] for d in generator]
# submit = pd.DataFrame(data={'ImageId':test_id, 'Label':output})
# submit.to_csv('nn_tensor_submit.csv'.format(name), index=False)

In [25]:
# neural network with keras
# TODO reshape input
# from keras.models import Sequential
# from keras.layers.core import Dense, Activation
# keras_model = Sequential([
#     Dense(input_dim=28*28, units=1000),
#     Activation('relu'),
#     Dense(units=10),
#     Activation('softmax'),
# ])
# keras_model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
# keras_model.fit(train_x.values, train_y.values, epochs=10, batch_size=100)
# loss, metrics = keras_model.evaluate(train_x.values, train_y.values)
# print(metrics)
# output = keras_model.predict_classes(test_x.values)
# print('\n', output[:10])
# submit = pd.DataFrame(data={'ImageId':test_id, 'Label':output})
# submit.to_csv('nn_keras_submit.csv'.format(name), index=False)

In [26]:
!head svm_submit.csv

ImageId,Label
1,2
2,0
3,9
4,9
5,3
6,7
7,0
8,3
9,0
